## Ignore

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("Data/S08_question_answer_pairs.txt", sep="	")

In [ ]:
df.sample(2)

In [ ]:
df.info()

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("Data/S08_question_answer_pairs.txt")
docs = loader.load()

In [ ]:
from os import sep
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="Data/S08_question_answer_pairs.csv", csv_args={
    'delimiter': ',',
    'quotechar': '"'})
data = loader.load()

In [ ]:
len(docs)

In [ ]:
len(data)

In [ ]:
type(docs)

In [ ]:
print(data)

___

## Use ChromaDB

In [14]:
import chromadb
import pandas as pd


In [15]:
df = pd.read_csv("Data/S08_question_answer_pairs.csv")
df.sample(2)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
1248,penguin,What order do penguins belong to?,Sphenisciformes,medium,medium,S08_set1_a3
1210,otter,What animals are related to otters?,"weasels, polecats, and badgers",hard,hard,S08_set1_a7


In [16]:
documents = []
metadatas = []
ids = []

for index, data in enumerate(df.iterrows()):
    # print(index)
    data_index, data = data
    # print(data['Question'])
    doc = "Question: "+str(data['Question'])+"\nAnswer: "+str(data['Answer'])
    documents.append(doc)
    metadatas.append({'source': data['ArticleTitle'], 'ArticleFile': data['ArticleFile']})
    ids.append(str(index + 1))

In [17]:
print(len(documents), len(metadatas), len(ids))

1715 1715 1715


In [18]:
chroma_client = chromadb.Client()
demo_collection = chroma_client.create_collection(name="demo_collection")

ValueError: Collection demo_collection already exists.

In [19]:
demo_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
Insert of existing embedding ID: 28
I

In [20]:
results = demo_collection.query(
    query_texts=["Who is older Calvin Jr. or John Coolidge?"],
    n_results=1
)

results

{'ids': [['234']],
 'distances': [[0.1776641607284546]],
 'metadatas': [[{'ArticleFile': 'S08_set3_a9', 'source': 'Calvin_Coolidge'}]],
 'embeddings': None,
 'documents': [['Question: Is Calvin Jr. older than John Coolidge?\nAnswer: No']]}

In [21]:
str(results["documents"][0][0])

'Question: Is Calvin Jr. older than John Coolidge?\nAnswer: No'

In [28]:
system_message = "You are a helpful assistant who answers the query based on the answer provided in the context below.\nContext: "+str(results["documents"][0][0])+str(results["metadatas"][0][0])+"Also provide your source"

In [29]:
# import os
import openai
openai.api_key = "OPENAIKEY"

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Who is older Calvin Jr. or John Coolidge?"}
  ]
)

print(completion.choices[0].message)


{
  "role": "assistant",
  "content": "John Coolidge is older than Calvin Jr."
}
